# Hybrid Search Deep Dive

Master hybrid search techniques with MongoDB `$rankFusion` and **MongoDB 8.2 Lexical Prefilters**.

**What you'll learn:**
- Vector vs keyword vs hybrid search
- Configuring fusion weights
- Understanding `$rankFusion` internals
- Per-pipeline score analysis
- **NEW: Lexical Prefilters** (fuzzy, phrase, wildcard, geo)
- Three Filter Systems architecture
- Search quality optimization

**Prerequisites:** Completed `01_getting_started.ipynb`

In [ ]:
# Setup
from dotenv import load_dotenv

from hybridrag import create_hybridrag

load_dotenv()
rag = await create_hybridrag()
print("✓ HybridRAG initialized")

## 1. Understanding Search Modes

In [ ]:
# Test query that benefits from different modes
query = "mongodb atlas vector database semantic search"

print(f"Query: '{query}'\n")

# Vector-only: Good for semantic similarity
vector_results = await rag.query(query=query, mode="vector", top_k=3)
print("VECTOR Mode (semantic similarity):")
for r in vector_results:
    print(f"  Score: {r.score:.4f} - {r.content[:60]}...")

print()

# Keyword-only: Good for exact term matching
keyword_results = await rag.query(query=query, mode="keyword", top_k=3)
print("KEYWORD Mode (exact matching):")
for r in keyword_results:
    print(f"  Score: {r.score:.4f} - {r.content[:60]}...")

print()

# Hybrid: Best of both worlds
hybrid_results = await rag.query(query=query, mode="hybrid", top_k=3)
print("HYBRID Mode (fusion):")
for r in hybrid_results:
    print(f"  Score: {r.score:.4f} - {r.content[:60]}...")

## 2. Configuring Fusion Weights

In [ ]:
query = "document embeddings vector similarity"

# Semantic-focused (80% vector, 20% keyword)
semantic_focused = await rag.query(
    query=query, mode="hybrid", vector_weight=0.8, text_weight=0.2, top_k=3
)

# Keyword-focused (20% vector, 80% keyword)
keyword_focused = await rag.query(
    query=query, mode="hybrid", vector_weight=0.2, text_weight=0.8, top_k=3
)

# Balanced (50/50)
balanced = await rag.query(
    query=query, mode="hybrid", vector_weight=0.5, text_weight=0.5, top_k=3
)

print("Semantic-focused (0.8/0.2):")
for r in semantic_focused:
    print(f"  {r.score:.4f} - {r.content[:50]}...")

print("\nKeyword-focused (0.2/0.8):")
for r in keyword_focused:
    print(f"  {r.score:.4f} - {r.content[:50]}...")

print("\nBalanced (0.5/0.5):")
for r in balanced:
    print(f"  {r.score:.4f} - {r.content[:50]}...")

## 3. Per-Pipeline Score Analysis

In [ ]:
# Query with score details
query = "atlas search full text"

results = await rag.query(query=query, mode="hybrid", top_k=3)

print(f"Query: '{query}'\n")
print("Per-Pipeline Scores:\n")

for idx, result in enumerate(results, 1):
    print(f"Result {idx}: {result.content[:50]}...")
    print(f"  Final Score: {result.score:.4f}")

    if hasattr(result, "source_scores") and result.source_scores:
        print(f"  Vector Score: {result.source_scores.get('vector', 'N/A')}")
        print(f"  Text Score: {result.source_scores.get('text', 'N/A')}")

    print()

## 4. Tuning for Query Types

In [ ]:
from hybridrag import detect_query_type

queries = [
    "What is MongoDB Atlas?",  # Factual - prefer keyword
    "explain vector embeddings concepts",  # Conceptual - prefer semantic
    "mongodb atlas vector search tutorial",  # Mixed - balanced
]

for query in queries:
    query_type = detect_query_type(query)

    # Adjust weights based on query type
    if "summary" in query_type.value:
        weights = (0.7, 0.3)  # More semantic
    elif "tool" in query_type.value:
        weights = (0.3, 0.7)  # More keyword
    else:
        weights = (0.5, 0.5)  # Balanced

    results = await rag.query(
        query=query,
        mode="hybrid",
        vector_weight=weights[0],
        text_weight=weights[1],
        top_k=2,
    )

    print(f"Query: {query}")
    print(f"Type: {query_type.value}")
    print(f"Weights: vector={weights[0]}, text={weights[1]}")
    print(f"Top result score: {results[0].score:.4f}\n")

## 5. Dynamic `numCandidates` Tuning

In [ ]:
# HybridRAG automatically calculates numCandidates = top_k * 20
# This ensures good recall

query = "mongodb vector search"

# Different top_k values
for top_k in [5, 10, 20]:
    results = await rag.query(query=query, mode="hybrid", top_k=top_k)

    print(f"top_k={top_k}:")
    print(f"  numCandidates={top_k * 20} (automatic)")
    print(f"  Results: {len(results)}")
    print(f"  Score range: {results[0].score:.4f} - {results[-1].score:.4f}\n")

## 6. Visualize Weight Impact

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

query = "mongodb atlas cloud database"
weight_configs = [(1.0, 0.0), (0.75, 0.25), (0.5, 0.5), (0.25, 0.75), (0.0, 1.0)]
labels = [
    "Vector\nOnly",
    "Semantic\nFocus",
    "Balanced",
    "Keyword\nFocus",
    "Keyword\nOnly",
]

avg_scores = []

for vec_w, text_w in weight_configs:
    mode = "vector" if vec_w == 1.0 else "keyword" if text_w == 1.0 else "hybrid"
    results = await rag.query(
        query=query, mode=mode, vector_weight=vec_w, text_weight=text_w, top_k=5
    )
    avg_scores.append(np.mean([r.score for r in results]))

plt.figure(figsize=(10, 6))
bars = plt.bar(
    labels, avg_scores, color=["#2196F3", "#4CAF50", "#FFC107", "#FF9800", "#F44336"]
)
plt.ylabel("Average Score")
plt.title(f'Search Quality Across Weight Configurations\nQuery: "{query}"')
plt.xticks(rotation=0)
plt.grid(axis="y", alpha=0.3)

# Add weight labels
for i, (vec_w, text_w) in enumerate(weight_configs):
    plt.text(
        i,
        avg_scores[i] + 0.01,
        f"{vec_w}/{text_w}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

## 7. Lexical Prefilters (MongoDB 8.2+)

**NEW in MongoDB 8.2:** Apply Atlas Search operators (fuzzy, phrase, wildcard, geo) **BEFORE** vector search.

This is a game-changer for search quality:
- `$vectorSearch` supports only MQL operators (`$eq`, `$gte`, `$in`)
- `$search.vectorSearch` supports full Atlas Search operators **as prefilters**

| Scenario | Legacy $vectorSearch | New $search.vectorSearch |
|----------|---------------------|--------------------------|
| "Find docs about *machin lerning*" | No fuzzy support | `fuzzy: {maxEdits: 2}` |
| "Exact phrase 'machine learning'" | Vector similarity only | `phrase: {slop: 0}` |
| "Tags matching tech*" | No wildcards | `wildcard: {query: "tech*"}` |
| "Docs within 10km of NYC" | No geo filtering | `geoWithin` |

In [ ]:
# Import lexical prefilter components
from hybridrag import (
    LexicalPrefilterConfig,
)

# Example 1: Fuzzy matching (typo-tolerant)
fuzzy_config = LexicalPrefilterConfig(
    fuzzy_filters=[
        {
            "path": "content",
            "query": "machin lerning",
            "maxEdits": 2,
        }  # Tolerates typos!
    ]
)

# Example 2: Exact phrase matching
phrase_config = LexicalPrefilterConfig(
    phrase_filters=[
        {"path": "content", "query": "vector database", "slop": 0}  # Exact phrase
    ]
)

# Example 3: Wildcard pattern matching
wildcard_config = LexicalPrefilterConfig(
    wildcard_filters=[
        {"path": "metadata.tags", "query": "mongo*"}  # Matches mongodb, mongoose, etc.
    ]
)

# Example 4: Combined filters
combined_config = LexicalPrefilterConfig(
    fuzzy_filters=[{"path": "content", "query": "search", "maxEdits": 1}],
    range_filters={"metadata.timestamp": {"gte": "2024-01-01"}},
    text_filters=[{"path": "metadata.category", "query": "technology"}],
)

print("Lexical Prefilter Configs Created:")
print(f"  Fuzzy: {fuzzy_config}")
print(f"  Phrase: {phrase_config}")
print(f"  Wildcard: {wildcard_config}")
print(f"  Combined: {combined_config}")

### Using Lexical Prefilters with Hybrid Search

In [ ]:
# Query with fuzzy prefiltering (typo-tolerant search)
query = "machine learning best practices"

# With lexical prefilter - tolerates typos in the prefilter
filter_config = LexicalPrefilterConfig(
    fuzzy_filters=[{"path": "content", "query": "machin lerning", "maxEdits": 2}],
    range_filters={"metadata.timestamp": {"gte": "2024-01-01"}},
)

# HybridRAG has lexical prefilters enabled by default (MongoDB 8.2+)
# Falls back gracefully to $vectorSearch on older MongoDB versions
results = await rag.query(
    query=query, mode="hybrid", lexical_filter_config=filter_config, top_k=5
)

print(f"Query: '{query}'")
print("Prefilters: fuzzy='machin lerning' (maxEdits=2), date>=2024-01-01")
print(f"\nResults ({len(results)} found):")
for idx, r in enumerate(results, 1):
    print(f"  {idx}. Score: {r.score:.4f} - {r.content[:60]}...")

### $meta Score Fields Reference (CRITICAL)

Different MongoDB operators use different `$meta` fields for scores:

| Operator | `$meta` Value | When Used |
|----------|---------------|-----------|
| `$vectorSearch` | `vectorSearchScore` | Legacy vector search |
| `$search.vectorSearch` | `searchScore` | MongoDB 8.2+ with lexical prefilters |
| `$rankFusion` | `rankFusionScore` | Hybrid search fusion |
| `$scoreFusion` | `scoreFusionScore` | Score-based fusion |

HybridRAG handles this automatically - you don't need to worry about the correct field.

In [ ]:
# Build the raw aggregation pipeline (for inspection)
from hybridrag import build_search_vector_search_stage

# Create a sample embedding (normally from Voyage AI)
sample_embedding = [0.1] * 1024  # Placeholder

# Build the $search.vectorSearch stage
pipeline_stage = build_search_vector_search_stage(
    query_vector=sample_embedding,
    lexical_filter_config=combined_config,
    vector_field="vector",
    index_name="vector_index",
    num_candidates=200,
    limit=10,
)

print("Generated $search.vectorSearch Pipeline Stage:")
print("-" * 50)
import json

print(json.dumps(pipeline_stage, indent=2, default=str))

## 8. Best Practices

### When to use each mode:

**Vector-only (semantic):**
- Conceptual questions
- Paraphrased queries
- Cross-lingual search

**Keyword-only (exact):**
- Technical terms
- Product names
- Error codes

**Hybrid (recommended):**
- Mixed queries
- Production systems
- When unsure

### Weight configuration tips:

- **Default (0.6/0.4)**: Good starting point
- **Semantic focus (0.7-0.8/0.2-0.3)**: For conceptual queries
- **Keyword focus (0.2-0.3/0.7-0.8)**: For exact matching
- **Test and iterate**: Use eval metrics

### Lexical Prefilters (MongoDB 8.2+):

- **Fuzzy filters**: For typo-tolerant searches (`maxEdits: 1-2`)
- **Phrase filters**: For exact multi-word matching (`slop: 0`)
- **Wildcard filters**: For pattern matching (`*`, `?`)
- **Geo filters**: For location-based filtering
- **Range filters**: For date/numeric ranges

### Performance:

- `numCandidates`: Automatic (top_k * 20)
- `scoreDetails: true`: Always enabled for debugging
- Monitor per-pipeline scores for optimization
- Lexical prefilters reduce candidate set BEFORE vectors (faster!)

## Next Steps

- `03_knowledge_graph_exploration.ipynb` - Graph-enhanced retrieval
- `04_prompt_engineering.ipynb` - Optimize for answer quality
- `05_performance_tuning.ipynb` - Production optimization

### Three Filter Systems Quick Reference

| Filter System | MongoDB Operator | Use Case |
|--------------|------------------|----------|
| `VectorSearchFilterConfig` | `$vectorSearch` | MQL operators (`$eq`, `$gte`, `$in`) |
| `AtlasSearchFilterConfig` | `$search.compound` | Atlas Search operators (`range`, `equals`) |
| `LexicalPrefilterConfig` | `$search.vectorSearch` | Atlas operators BEFORE vectors (fuzzy, phrase, wildcard, geo) |